# TensorFlow for ICT Applications

## Lab session 2: Linear regression

**Content of this lab session:**

1. Installing TensorFlow 2.0.
2. Defining and manipulating tensors (with NumPy interoperability).
3. Automatic differentiation in TensorFlow.
4. A simple linear regression example.

### Installing TensorFlow 2.0

In [1]:
# Need the alpha release until official release
!pip install tensorflow==2.0.0-alpha0

^C


In [1]:
# Check version
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [2]:
# This should be true at startup - otherwise, you are running an old version
tf.executing_eagerly()

True

In [3]:
# Check if a GPU is available on the computer
tf.test.is_gpu_available()

False

### Defining tensors and operating with NumPy

In [4]:
x = tf.constant(2.0)
print(x) # Check the default type

tf.Tensor(2.0, shape=(), dtype=float32)


In [5]:
# Casting operation
x = tf.cast(x, tf.float64)

In [6]:
# Full interoperability with NumPy
import numpy as np
y = np.array(3)
c = x + y
print(c)

tf.Tensor(5.0, shape=(), dtype=float64)


In [7]:
c = x.numpy() + y # Result is now a NumPy array
print(c)

5.0


In [8]:
# Matrix definition
X = tf.random.uniform((4, 3))

In [9]:
# Similar indexing as in NumPy
X[0]

<tf.Tensor: id=17, shape=(3,), dtype=float32, numpy=array([0.10559726, 0.17178488, 0.6278167 ], dtype=float32)>

In [10]:
# Equivalent to X.sum() in NumPy
tf.reduce_sum(X)

<tf.Tensor: id=20, shape=(), dtype=float32, numpy=4.4616857>

### Automatic differentiation with TensorFlow

In [11]:
# Define a simple function working on TF tensors
def J(w):
    return 2.0 * tf.cos(w)

In [12]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = J(x)

In [13]:
# Automatic differentiation
ygrad = tape.gradient(y, x)

In [14]:
ygrad

<tf.Tensor: id=38, shape=(), dtype=float64, numpy=-1.8185948536513634>

In [15]:
# Check the gradient is numerically correct
ygrad.numpy() == -2.0*np.sin(x.numpy())

True

In [16]:
# Variables are automatically watched
v = tf.Variable(tf.random.uniform((4, 3)))
with tf.GradientTape() as tape:
    y = J(v)

In [17]:
ygrad = tape.gradient(y, v)
print(ygrad)

tf.Tensor(
[[-0.719849   -0.22737494 -1.3421247 ]
 [-1.5472176  -1.2552158  -0.78375834]
 [-0.35114276 -0.3001388  -0.07388174]
 [-1.2883877  -1.6333557  -1.1794277 ]], shape=(4, 3), dtype=float32)


In [18]:
# Let's try with two variables
v1 = v
v2 = tf.Variable(3.0)

In [19]:
# Numerical functions can also have if/while control loops
is_cos = True
with tf.GradientTape() as tape:
  y = J(v1)
  if is_cos:
    y = y * tf.cos(v2)
  else:
    y = y * tf.sin(v2)

In [20]:
g = tape.gradient(y, [v1, v2])

In [21]:
g

[<tf.Tensor: id=119, shape=(4, 3), dtype=float32, numpy=
 array([[0.7126451 , 0.22509949, 1.3286934 ],
        [1.5317339 , 1.2426542 , 0.7759149 ],
        [0.3476287 , 0.29713517, 0.07314236],
        [1.2754941 , 1.6170099 , 1.1676246 ]], dtype=float32)>,
 <tf.Tensor: id=106, shape=(), dtype=float32, numpy=-2.856872>]

In [22]:
# Higher-order gradients
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        y = J(v)
    ygrad = inner_tape.gradient(y, v)
ygradgrad = outer_tape.gradient(ygrad, v)

In [23]:
ygradgrad.numpy()

array([[-1.8659629, -1.9870331, -1.4828019],
       [-1.2673271, -1.5570592, -1.8400334],
       [-1.9689335, -1.977351 , -1.9986349],
       [-1.5297246, -1.1541877, -1.6152245]], dtype=float32)

## Linear regression with automatic differentiation

In [ ]:
# Define the data
import numpy as np
N = 25
# y = wx + b + epsilon
w = -0.3
b = 0.5
X = np.random.rand(N)
y = X * w + b + np.random.randn(N)*0.2

In [ ]:
# Some initial plot
import matplotlib.pyplot as plt
plt.figure()
plt.scatter(X, y)
plt.show()

In [ ]:
import tensorflow as tf
tf.__version__ # Should be '2.0.0-alpha0'

In [ ]:
# Cast the data to TF tensors
X = tf.constant(X)
y = tf.constant(y)

In [ ]:
# Cast the data to tf.float32 for compatibility with default in tf.Variable
X = tf.cast(X, tf.float32)
y = tf.cast(y, tf.float32)

In [ ]:
# Random initial guess for our linear regression parameters
west = tf.Variable(0.05)
best = tf.Variable(0.0)

In [ ]:
def linear(xb):
    # Output of the linear model
    return xb * west + best

In [ ]:
# Check it is returning predictions
linear(X)

In [ ]:
# We will use this to save loss evolution and w's gradient throughout iterations
all_losses = []
all_grads = []

In [ ]:
for i in range(1000):
    
    with tf.GradientTape() as tape:
    
        # Compute the output of the model
        ypred = linear(X)
    
        # Compute the error of the model
        l = tf.reduce_mean((ypred - y)**2)
        all_losses.append(l.numpy())
    
    # Compute the gradients of the error
    g = tape.gradient(l, [west, best])
    all_grads.append(g[0].numpy())
    
    # Optimize the parameters using the gradients
    west.assign_add(- 0.01 * g[0])
    best.assign_add(- 0.01 * g[1])

In [ ]:
# Plot the loss
plt.figure()
plt.plot(all_losses)

In [ ]:
# Plot the gradients of w
plt.figure()
plt.plot(all_grads)

In [ ]:
# Plot the final predictions
plt.figure()
plt.scatter(X, y)
plt.plot(X, linear(X))

In [ ]:
# Check the values of the parameters
print(west)
print(best)